In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# Use ensemble to train the model
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import lightgbm as lgb
import sys
import sys

# In the second part of the notebook
from sklearn.preprocessing import LabelEncoder

# This part of importing would be moved to main.py or train.py later
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

from scipy.stats import zscore
from scipy.special import boxcox1p

import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
# from catboost import CatBoostRegressor

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import wandb

Define a hyperparameter class 

In [ ]:
class Config:
    def __init__(self):
        self.objective = 'regression'
        self.num_leaves = 31
        self.learning_rate = 0.01
        self.n_estimators = 120
        self.max_bin = 255
        self.bagging_fraction = 0.8
        self.bagging_freq = 5
        self.feature_fraction = 0.8
        self.feature_fraction_seed = 9
        self.bagging_seed = 9
        self.random_state = 42
        self.OrgFertilizers = 'None'
        self.FirstTopDressFert = 'None' 
        self.CropbasalFerts = 'None'
        self.NursDetFactor = 'None'
        self.LandPreparationMethod = 'None'
        self.TransDetFactor = 'None'
        self.PCropSolidOrgFertAppMethod = 'None'
        self.sparse_threshold = 1.0
# Instantiate the Hyperparameters class
config = Config()

# Initialize a wandb run
wandb.init(project='zindi-crop-challenge',
           entity="lmu-seminar")

# Update the wandb configuration with the hyperparameters
wandb.config.update(vars(config))
    

In [2]:
# Read the dataset
path = 'Train.csv'
dataset = pd.read_csv(path)

In [3]:
test_path = 'Train.csv'
dataset_test = pd.read_csv(test_path)

In [4]:
# Save the 'Yield' column
train_labels = dataset['Yield'].copy()
dataset = dataset.drop(columns=['ID','Yield'], axis = 1)
dataset_test = dataset_test.drop(columns=['ID'], axis = 1)

# Copy the original dataset
dataset_original = dataset.copy()
dataset_test_original = dataset_test.copy()

In [5]:
dataset['Harv_hand_rent'] = dataset['Harv_hand_rent'].fillna(0)
dataset['OrgFertilizers'] = dataset['OrgFertilizers'].fillna(dataset['OrgFertilizers'].mode()[0])
dataset['CropbasalFerts'] = dataset['CropbasalFerts'].fillna(dataset['CropbasalFerts'].mode()[0])
dataset['FirstTopDressFert'] = dataset['FirstTopDressFert'].fillna(dataset['FirstTopDressFert'].mode()[0])
dataset['NursDetFactor'] = dataset['NursDetFactor'].fillna(dataset['NursDetFactor'].mode()[0])
dataset['LandPreparationMethod'] = dataset['LandPreparationMethod'].fillna(dataset['LandPreparationMethod'].mode()[0])
dataset['TransDetFactor'] = dataset['TransDetFactor'].fillna(dataset['TransDetFactor'].mode()[0])
dataset['PCropSolidOrgFertAppMethod'] = dataset['PCropSolidOrgFertAppMethod'].fillna(dataset['PCropSolidOrgFertAppMethod'].mode()[0])

dataset_test['Harv_hand_rent'] = dataset_test['Harv_hand_rent'].fillna(0)
dataset_test['OrgFertilizers'] = dataset_test['OrgFertilizers'].fillna(dataset_test['OrgFertilizers'].mode()[0])
dataset_test['CropbasalFerts'] = dataset_test['CropbasalFerts'].fillna(dataset_test['CropbasalFerts'].mode()[0])
dataset_test['FirstTopDressFert'] = dataset_test['FirstTopDressFert'].fillna(dataset_test['FirstTopDressFert'].mode()[0])
dataset_test['NursDetFactor'] = dataset_test['NursDetFactor'].fillna(dataset_test['NursDetFactor'].mode()[0])
dataset_test['LandPreparationMethod'] = dataset_test['LandPreparationMethod'].fillna(dataset_test['LandPreparationMethod'].mode()[0])
dataset_test['TransDetFactor'] = dataset_test['TransDetFactor'].fillna(dataset_test['TransDetFactor'].mode()[0])
dataset_test['PCropSolidOrgFertAppMethod'] = dataset_test['PCropSolidOrgFertAppMethod'].fillna(dataset_test['PCropSolidOrgFertAppMethod'].mode()[0])

In [6]:
# Checking the unique values for categorical variables to identify sparse classes
categorical_columns = dataset.select_dtypes(include=['object']).columns
sparse_classes = {col: dataset[col].nunique() for col in categorical_columns if dataset[col].nunique() > 15} 

In [7]:
# check the name of sparse classes
print(sparse_classes.keys())

dict_keys(['LandPreparationMethod', 'CropTillageDate', 'RcNursEstDate', 'SeedingSowingTransplanting', 'NursDetFactor', 'TransDetFactor', 'OrgFertilizers', 'CropbasalFerts', 'Harv_date', 'Threshing_date'])


In [8]:
# check the unique values of sparse classes
print(sparse_classes.values())

dict_values([43, 78, 61, 62, 125, 155, 31, 34, 107, 162])


In [9]:
# Define a threshold for grouping
threshold_percentage = 1
threshold = len(dataset) * (threshold_percentage / 100)

In [10]:
# Function to group sparse classes
def group_sparse_classes(df, column, threshold):
    # Find the categories that are below the threshold
    value_counts = df[column].value_counts()
    to_replace = value_counts[value_counts <= threshold].index.tolist()
    
    # Replace the sparse classes with 'other'
    df[column] = df[column].replace(to_replace, 'other')
    return df

In [11]:
# Apply grouping for the identified categorical variables with many unique values
for col in ['LandPreparationMethod', 'OrgFertilizers', 'CropbasalFerts']:
    dataset = group_sparse_classes(dataset, col, threshold)

In [12]:
# Apply the category adjustment to the test dataset based on the training dataset categories
for col in ['LandPreparationMethod', 'OrgFertilizers', 'CropbasalFerts']:
    test_dataset = group_sparse_classes(dataset_test, col, threshold)

In [13]:
# fill the high skewness columns with median
# check the skewness of the numerical columns
skewness = dataset.skew(numeric_only=True)
skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

# sort the skewness in descending order
skewness.sort_values(ascending=False)
print(skewness)

skewed_features = skewness.index
lam = 0.15
# apply the boxcox1p transformation to the skewed features
for feat in skewed_features:
    dataset[feat] = boxcox1p(dataset[feat], lam)
    dataset_test[feat] = boxcox1p(dataset_test[feat], lam)

# check the skewness of the numerical columns
skewness = dataset.skew(numeric_only=True)
skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

There are 18 skewed numerical features to Box Cox transform
CultLand                         8.178796
CropCultLand                     8.796317
SeedlingsPerPit                 53.528364
TransplantingIrrigationHours    30.754264
TransIrriCost                    3.912326
StandingWater                    1.848641
Ganaura                          5.237242
CropOrgFYM                       6.876849
BasalDAP                         2.905715
BasalUrea                        2.244465
1tdUrea                          2.094479
1appDaysUrea                     4.769872
2tdUrea                          2.696580
2appDaysUrea                    -1.589283
Harv_hand_rent                  40.547049
Residue_length                  -1.880371
Residue_perc                     3.747956
Acre                             2.384037
dtype: float64
There are 11 skewed numerical features to Box Cox transform


In [16]:
high_skewed_features = ['SeedingsPerpit', 'TransplantingIrrigationHours','Harv_hand_rent']
low_skewed_features = ['CultLand', 'CropOrgFYM', 'CropCultLand','Ganaura'] 
moderate_skewed_features = ['BasalUrea', 'BasalDAP',
                            '1tdUrea', '1appDaysUrea', 
                            '2tdUrea', '2appDaysUrea',
                            'Residue_length', 'Residue_perc',
                            'StandingWater', 'TransIrriCost']

In [17]:
# Fill the low skewness columns with mean
for feat in low_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].mean())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].mean())

# Fill the moderate skewness columns with median
for feat in moderate_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].median())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].median())

# Fill the high skewness columns with median
for feat in high_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].median())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].median())

TypeError: can only concatenate str (not "float") to str

In [ ]:
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        dataset[col] = dataset[col].astype('category').cat.codes
    elif dataset[col].dtype == 'datetime64[ns]':
        dataset = dataset.drop(columns=[col])

for col in dataset_test.columns:
    if dataset_test[col].dtype == 'object':
        dataset_test[col] = dataset_test[col].astype('category').cat.codes
    elif dataset_test[col].dtype == 'datetime64[ns]':
        dataset_test = dataset_test.drop(columns=[col])

In [ ]:
X = dataset
y = train_labels    

# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=200)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Select the object columns and categorical columns
object_cols = X_train.select_dtypes(include=['object'])
categorical_cols = X_train.select_dtypes(include=['category'])
whole_categorical_cols = pd.concat([object_cols, categorical_cols], axis=1) 
# generate a list with the name of the categorical columns
whole_categorical_cols = whole_categorical_cols.columns
# Transfer into a list
whole_categorical_cols = list(whole_categorical_cols)

In [ ]:
# Parameters
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=82)

# Prepare an array to store the RMSE for each fold
rmse_scores = []
models = []

# Initialize an empty array to hold feature importances
feature_importances = np.zeros(X_train.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    # Split the data
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a LGBMRegressor object
    lgbm_model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.01, n_estimators=100)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=lgbm_model.best_iteration_)

    # Calculate and print RMSE for the current fold
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    print(f"Fold {fold}: RMSE: {fold_rmse}")

    # Accumulate feature importances
    feature_importances += lgbm_model.feature_importances_

    models.append(lgbm_model)

# After cross-validation, print the mean RMSE
print(f"Mean RMSE: {np.mean(rmse_scores)}, STD RMSE: {np.std(rmse_scores)}")

# Feature importances from all folds
feature_importances = feature_importances / n_splits

test_predictions = []

for model in models:
    # Make predictions
    fold_preds = model.predict(dataset_test, num_iteration=model.best_iteration_)
    test_predictions.append(fold_preds)

# Now average these predictions
test_predictions = np.column_stack(test_predictions)
y_pred_test = np.mean(test_predictions, axis=1)

# Now you have `y_pred_test` which is the averaged predictions from all folds

# Create a submission file
dataset_upload = dataset_test_original

submission_df = pd.DataFrame({'ID': dataset_upload['ID'], 'Yield': y_pred_test})
submission_df.to_csv('submission_07_11_567.csv', index=False)

In [ ]:
# Initialize a new wandb run
wandb.init(project='zindi-crop-challenge', 
           entity="lmu-seminar")

# Define your hyperparameters
hyperparameters = dict(
    objective='regression',
    num_leaves=31,
    learning_rate=0.009,
    n_estimators=120,
    # Add other hyperparameters you want to track here
)

# Save hyperparameters to wandb
wandb.config.update(hyperparameters)

# Rest of your code for preparing the data
# ...

# Parameters for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=82)

rmse_scores = []
models = []

feature_importances = np.zeros(X_train.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a LGBMRegressor object with hyperparameters from wandb
    lgbm_model = lgb.LGBMRegressor(**wandb.config)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols,
        callbacks=[wandb.callback()]
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=lgbm_model.best_iteration_)

    # Calculate RMSE and log the metric to wandb
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    wandb.log({'fold_rmse': fold_rmse, 'fold': fold})

    # Feature importances
    feature_importances += lgbm_model.feature_importances_

    # Save the model to wandb
    wandb.sklearn.log_model(lgbm_model, 'LGBMRegressor')
    
    models.append(lgbm_model)

# Log mean and standard deviation of RMSE across folds to wandb
wandb.log({'mean_rmse': np.mean(rmse_scores), 'std_rmse': np.std(rmse_scores)})

# Feature importances
feature_importances = feature_importances / n_splits
# You can also log feature importances to wandb here

# Finalize and close your wandb run
wandb.finish()